# LSST Solar System Processing

## Linking of Observations from JPL Veres & Chesley dataset via HelioLinC

### Algorithm: 

Based on HelioLinC (Holman et al. 2018) we transform topocentric observations to heliocentric states assuming a distance and radial velocity.
The resulting 3D positions are collected into tracklets. Tracklets contain at least two observations and can, thus, be used to create velocity vectors.
A tracklet + velocity vector is called an "arrow". 
Arrows are propagated to a common epoch using spiceypy's 2body propagator, and then clustered using dbscan.

### Implementation:
S. Eggl 20191215
    

In [1]:
# Solar System Processing functions
import lsstssp as ls

In [2]:
import constants as cnst
import vector as vec
import transforms as tr
import propagate as prop
import ephemeris as ephem
import state as st

In [3]:
#import thor
#from thor.orbits import iod

In [4]:
# Gauss method for initial orbit determination
#import gauss2 as iod

In [5]:
#Accelerators
import numpy as np
#import numba

#Database
import pandas as pd
import sqlite3 as sql

# # External API's
# from astroquery.jplhorizons import Horizons
from astroquery.jplsbdb import SBDB

#Orbital Dynamics
import spiceypy as sp

#Interpolation
import scipy.interpolate as spi

# Clustering
import scipy.spatial as scsp
import sklearn.cluster as cluster

#Plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

import matplotlib
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 12}

matplotlib.rc('font', **font)

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d

import plotly.graph_objects as go
import seaborn as sns
sns.set_context('poster')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.25, 's' : 40, 'linewidths':0}

#Timing
import time
#%matplotlib inline


/epyc/projects/pytrax_play/envs/pytrax_s/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [6]:
duende_state=np.array([   6.832993744130379E-01, -7.578357776615274E-01,  4.887250343702505E-02,
   1.115044267163202E-02,  1.227682492646761E-02, -2.983535088629754E-03])
epoch=2456165.500000000

In [7]:
help(tr)

Help on module transforms:

NAME
    transforms - transforms

DESCRIPTION
    LSST Solar System Processing
    
    Transformations between common coordinate and time systems
    Implementation: Python 3.6, S. Eggl 20191115

FUNCTIONS
    cartesian2cometary(epoch, state, frame='ecliptic', mu=0.0002959122082326087)
        Spiceypy conversion from heliocentric
        cartesian states to cometary orbital elements.
        
        Parameters:
        -----------
        epoch ... epoch of orbital elements [time units]
        state ... heliocentric ecliptic cartesian state (x,y,z,vx,vy,vz)
        frame ... Coordinate frame of Cartesian states: 'ecliptic', 'icrf'
        mu    ... Gravitational parameter
        
        Returns:
        --------
        com ... orbital elements array
                q    = pericenter distance
                e    = eccentricity
                i    = inclination (deg)
                node = longitude of the ascending node (deg)
                w    = a

In [8]:
tr.cartesian2cometary(epoch, duende_state)

265.5868149404234


([0.8934926248531639,
  0.10812935216147165,
  10.337299584234758,
  147.26247924376153,
  271.08626138837343,
  2455895.2995374575],
 2456165.5,
 366.2537484667051)

In [9]:
# Initial IAU76/J2000 heliocentric ecliptic osculating elements (au, days, deg.):
#   EPOCH=  2456165.5 ! 2012-Aug-26.00 (TDB)         Residual RMS= .28233        
#    EC= .10812935219311     QR= .8934926246676751   TP= 2456261.5532859839      
#    OM= 147.2624792437615   W=  271.086261479318    IN= 10.33729958423475       
#   Equivalent ICRF heliocentric equatorial cartesian coordinates (au, au/d):
#    X= 6.832993744130375E-01  Y=-7.147410974191529E-01  Z=-2.566101150697677E-01
#   VX= 1.115044267163202E-02 VY= 1.245054875137604E-02 VZ= 2.146100577752526E-03

In [10]:
jpl_data_path="/data/epyc/data/solarsystem/jpl/fullDensity_3months/jpl_fullDensity.db"

def grab_JPL_data(database,nrows):
    """Import JPL LSST Veres & Chesley dataset from local database
    
    Parameters:
    -----------
    database ... path to database
    nrows ... number of rows 
    
    Returns:
    --------
    observations ... pandas dataframe containing observations from JPL database
    """
    con = sql.connect(database)
    observations = pd.read_sql("""SELECT * FROM detections LIMIT """+str(nrows), con)
    
    return observations


def grab_n_days_of_JPL_data(database,tstart,tend):
    """Import JPL LSST Veres & Chesley dataset from local database
    
    Parameters:
    -----------
    database ... path to database
    tstart ... start night: number of nights since start of LSST survey (kraken2026) 
    tend ... last night: number of nights since start 
    
    Returns:
    --------
    observations ... pandas dataframe containing observations from JPL database
    """
    t0=52390
    
    if (tstart>tend):
        print('Start night must be before end night! ')
    
    else:
        qnights="nn <= " + str(t0+tend) + " AND nn >= " + str(t0+tstart)
        print(qnights)
        con = sql.connect(database)
        observations = pd.read_sql("""SELECT * FROM detections WHERE """+qnights, con)
    
    return observations
    

In [11]:
#obs=grab_n_days_of_JPL_data(jpl_data_path,0,7)

In [12]:
#obs

In [13]:
def plot_clusters(data, algorithm, args, kwds):
    """Cluster and plot data with skit learn algorithm """
    #start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(data)
    #end_time = time.time()
    palette = sns.color_palette('deep', np.unique(labels).max() + 1)
    plt.figure(dpi=300)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    plt.scatter(data.T[0], data.T[1], c=colors, **plot_kwds)
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    plt.title('Clusters found by {}'.format(str(algorithm.__name__)), fontsize=24)
    #plt.text(-0.5, 0.7, 'Clustering took {:.2f} s'.format(end_time - start_time), fontsize=14)

In [14]:
def plot_clusters3d(data, dims, algorithm, args, kwds):
    """Cluster and plot multi dimensional data with skit learn algorithm
        Plotting is done in 3D only,"""
    #start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(data)
    #end_time = time.time()
    palette = sns.color_palette('deep', np.unique(labels).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    
    fig=plt.figure(dpi=300,figsize=(20,14))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter3D(data.T[dims[0]], data.T[dims[1]], data.T[dims[2]], c=colors)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    frame = plt.gca()
    #frame.axes.get_xaxis().set_visible(False)
    #frame.axes.get_yaxis().set_visible(False)
    
    plt.title('Clusters found by {}'.format(str(algorithm.__name__)), fontsize=22)
    #plt.text(-0.5, 0.7, 'Clustering took {:.2f} s'.format(end_time - start_time), fontsize=10)

In [15]:
def plot_orbits3d(df, xlims, ylims, zlims):
    """Orbit plot in 3D,"""
    palette = sns.color_palette('deep', np.unique(df['objId']).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in df['objId']]
    
    fig=plt.figure(dpi=300,figsize=(20,14))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter3D(df['x'], df['y'], df['z'], c=colors)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    frame = plt.gca()

    if(xlims != []):
        ax.set_xlim(xlims[0],xlims[1])
    if(ylims != []):
        ax.set_ylim(ylims[0],ylims[1])
    if(zlims != []):
        ax.set_zlim(zlims[0],zlims[1])    
    
    plt.title('Orbits', fontsize=22)
    #plt.text(-0.5, 0.7, 'Clustering took {:.2f} s'.format(end_time - start_time), fontsize=10)

In [16]:
def plot_positions3d(t,x,y,z,*args,**kwargs):
    """Plot x,y,z positions in 3D with color coding (time).
    
    Parameters:
    -----------
    t ... time array
    x ... array containing x coordinates
    y ... array containing t coordinates
    z ... array containing z coordinates
    
    Kwargs:
    -------
    xlim, ylim, zlim ... [min,max] ranges for x,y and z axes
    """
#t = dt[:]
#x, y, z = xp[:,0], xp[:,1], xp[:,2]

    options = {'xlim' : [], 'ylim' : [] }

    options.update(kwargs)

    data=[go.Scatter3d(x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=3,
            color=t,                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.8
                    ))]

    layout = go.Layout(
        xaxis=dict(range=options['xlim']),
        yaxis=dict(range=options['ylim'])
 #       zaxis=dict(range=options['zlim'])
                  )

    fig = go.Figure(data=data, layout=layout)

# tight layout
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

In [17]:
# def metrics(df,pairs,cluster):
#     """Metrics for the succes of LSSTSSP arrow generation and clustering of SSO observations.
    
#     Parameters:
#     -----------
#     df      ... pandas dataframe with observations
#     pairs   ... list of pairs of observations [obsid1,obsid2] linked into arrows
#     cluster ... output of clustering algorithm (sklearn.cluster)
    
#     Returns:
#     --------
#     n_obj               ... number of SSOs/asteroids in observations file 
#     n_clusters          ... number of clusters found by clustering algorithm (including garbage cluster with index -1)
#     perfect_clusters    ... fraction of perfect clusters that contain all observations of one object only 
#     pure_clusters       ... fraction of clusters that contain observations of one object 
#     impure_clusters     ... fraction of clusters that contain mised observations (more than one object)     
#     completeness        ... fraction of objects discovered
#     obj_not_found       ... list of objects not found 
#     unique_labels       ... unique cluster labels 
#     obj_in_cluster      ... list of objects in per cluster 
#     obs_in_cluster      ... list of observations in cluster
#     """
#     #cluster names (beware: -1 is the cluster of all the leftovers)
#     unique_labels = np.unique(cluster.labels_)
#     #number of clusters
#     n_clusters = len(unique_labels)
    
#     #which objects do observations in pairs (tracklets) belong to
#     p = np.array(pairs)              
#     pair_obj = np.array([df['obj'][p[:,0]].values,df['obj'][p[:,1]].values]).T
#     #which pairs (tracklets) belong to the same objects?
#     correct = correct_pairs(df,pairs)
#     #if a pair is correct pair_ok==1 otherwise 0
#     pair_ok = np.zeros(len(cluster.labels_))
#     pair_ok[correct] = 1

#     #group original observation dataframe by objects
#     gdf=df.groupby(['obj'])
    
#     #cluster contains all observations of one object
#     perfect_clusters =0 
#     #cluster contains observations of one object only
#     pure_clusters = 0
# #     #cluster contains all observations of one object + pollution
# #     valid_clusters = 0
#     #cluster contains all observations of more than one object
#     impure_clusters = 0
    
#     obs_in_cluster=[]
#     obs_in_cluster_add=obs_in_cluster.append
    
#     obj_in_cluster=[]
#     obj_in_cluster_add=obj_in_cluster.append
#     #cluster contains 
#     for u in unique_labels:
#         #which indices in pair array appear in a given cluster?
#         idx = np.where(cluster.labels_ == u)[0]
#         #find unique object ids in cluster
#         uniq_obj=np.unique(pair_obj[idx])
        
#         obj_in_cluster_add(uniq_obj)
        
#         #which observations are in this cluster
#         obs_in_cluster_add(np.unique(p[idx].flatten()))
        
#         if (np.sum(pair_ok[idx]) == len(idx)):
#             perfect_clusters += 1
#         elif (len(uniq_obj) == 1):
#             pure_clusters += 1 
#         elif (len(uniq_obj) > 1):
#             impure_clusters += 1 
        
#         if(uniq_obj)
         
        
#             #find unique observation ids in cluster
#             uniq_obs=np.rint(np.unique(p[idx]))
      
#             #find observation ids linked with objects
# #             valid=False
# #             for obj in uniq_obj:
# #                 #print(int(obj))
# #                 #obs_idx=gdf.indices[int(obj)+1]
# #                 obs_obj=gdf.indices[obj]
# #                 #print('obs_obj',obs_obj)
# #                 #print('uniq_obs',uniq_obs)
# # #                 print('***************')
# # #                 print('cluster',u,'obj', obj)
# # #                 print(np.isin(obs_obj,uniq_obs))
# # #                 print(all(np.isin(obs_obj,uniq_obs)))
# #                 if(all(np.isin(obs_obj,uniq_obs))):
# #                     valid=True
# #             if(valid):
# #                 valid_clusters += 1    
#     #number of unique found objects in clusters vs number of actual objects
#     #print((np.hstack(obj_in_cluster[1:])))
#     n_obj=len(gdf)
#     uoc=np.unique(np.hstack(obj_in_cluster[1:]))
#     completeness=len(uoc)/n_obj*100
#     #objects not found
#     obj_in_sample=list(gdf.indices.keys())
#     obj_not_found=np.ma.array(obj_in_sample, mask=np.isin(obj_in_sample,uoc))
    
#     return [n_obj, n_clusters, perfect_clusters, pure_clusters, impure_clusters, completeness, obj_not_found, unique_labels, obj_in_cluster, obs_in_cluster]                     


In [18]:
def correct_pairs(df,pairs):
    """Which pairs are actually good, i.e. which tracklets are real?
    
    Parameters:
    -----------
    df      ... pandas dataframe with observations
    pairs   ... list of pairs of observations [obsid1,obsid2] linked into arrows
    
    Returns:
    --------
    correct ... logical array (dimension of pairs[:,0])
    """
    p=np.array(pairs)
    #find out which pairs are actually good
    pair_obj=np.array([df['objId'][p[:,0]].values,df['objId'][p[:,1]].values]).T
    #print(pair_obj)
    #correct=np.where(df['obj'][p[:,0]].values == df['obj'][p[:,1]].values)
    correct=np.where(pair_obj[:,0] == pair_obj[:,1])
    return correct[0]

In [19]:
def observations_in_cluster(df,pairs,cluster, garbage=False):
    """List observations in each cluster.
    
    Parameters:
    -----------
    df      ... pandas dataframe with observations
    pairs   ... list of pairs of observations [obsid1,obsid2] linked into arrows
    cluster ... output of clustering algorithm (sklearn.cluster)
    
    Returns:
    --------
    obs_in_cluster ... list of observations in each cluster
    """
    #cluster names (beware: -1 is the cluster of all the leftovers)
    if(garbage):
        unique_labels = np.unique(cluster.labels_)
    else:
        unique_labels = np.unique(cluster.labels_)[1:]
    #number of clusters
    n_clusters = len(unique_labels)
    
    #which objects do observations in pairs (tracklets) belong to
    p = np.array(pairs)              
    
    obs_in_cluster=[]
    obs_in_cluster_add=obs_in_cluster.append
    
    #cluster contains 
    for u in unique_labels:
        #which indices in pair array appear in a given cluster?
        idx = np.where(cluster.labels_ == u)[0]
        
        #which observations are in this cluster
        obs_in_cluster_add(np.unique(p[idx].flatten()))
    
    return obs_in_cluster, unique_labels

In [20]:
def objects_in_cluster(df,pairs,cluster):
    """List observations in each cluster.
    
    Parameters:
    -----------
    df      ... pandas dataframe with observations
    pairs   ... list of pairs of observations [obsid1,obsid2] linked into arrows
    cluster ... output of clustering algorithm (sklearn.cluster)
    
    Returns:
    --------
    obs_in_cluster ... list of objects in each cluster
    """
    #cluster names (beware: -1 is the cluster of all the leftovers)
    unique_labels = np.unique(cluster.labels_)
    #number of clusters
    n_clusters = len(unique_labels)
    
    #which objects do observations in pairs (tracklets) belong to
    p = np.array(pairs)              
    pair_obj = np.array([df['obj'][p[:,0]].values,df['obj'][p[:,1]].values]).T
    
    obj_in_cluster=[]
    obj_in_cluster_add=obj_in_cluster.append
    #cluster contains 
    for u in unique_labels:
        #which indices in pair array appear in a given cluster?
        idx = np.where(cluster.labels_ == u)[0]
        #find unique object ids in cluster
        uniq_obj=np.unique(pair_obj[idx])
        obj_in_cluster_add(uniq_obj)
        
    return obj_in_cluster, unique_labels

In [21]:
def observations_in_arrows(df,goodpairs,*args,**kwargs):
    """Find which observations go into an arrow.
    
    Parameters:
    -----------
    df          ... pandas dataframe with observations
    goodpairs   ... filtered list of pairs of observations [obsid1,obsid2] linked into arrows
    
    Returns:
    --------
    df_obs_in_arrows ... pandas dataframe where the index is the arrow id 
                         and the first and second column are the first and second observation 
                         that go into the respective arrow.
    """
    df_obs_in_arrows=pd.DataFrame(goodpairs,**kwargs)
    return df_obs_in_arrows

In [22]:
def df2difi(df,index_name,value_name):
    """Map pandas dataframe with lists of values to THOR difi format"""

    difi=df[value_name].apply(pd.Series) \
    .merge(df, right_index = True, left_index = True) \
    .drop([value_name], axis = 1) \
    .melt(id_vars = [index_name], value_name = value_name) \
    .drop("variable", axis = 1) \
    .dropna() \
    .sort_values(by=[index_name]) \
    .astype('int') \
    .reset_index(drop=True)
    
    return difi

## SIMULATED OBSERVATIONS 


1) Select observations from JPL Dataset

In [23]:
%%time
dfjpl=grab_n_days_of_JPL_data(jpl_data_path,0,6)

nn <= 52396 AND nn >= 52390
CPU times: user 1min 44s, sys: 38.8 s, total: 2min 23s
Wall time: 1min 40s


In [24]:
def obs2heliocentric_arrows(df, r, drdt, tref, lttc=False, verbose=True):
    """Create tracklets/arrows from dataframe containing nightly RADEC observations
       and observer positions.

    Parameters:
    -----------
    df       ... Pandas DataFrame containing nightly RA and DEC [deg], time [JD, MJD],
                 (x,y,z)_observer positions [au, ICRF]
    r        ... assumed radius of heliocentric sphere used for arrow creation[au]
    drdt     ... assumed radial velocity
    tref     ... reference time for arrow generation. Used to calculate how much the 
                 heliocentric distance changes between observations based on assumed dr/dt


    Keyword arguments:
    ------------------
    lttc (optional)        ... light travel time correction
    verbose (optional)     ... print verbose progress statements  

    Returns:
    --------
    x         ... tracklet/arrow position (3D) [au]
    y         ... tracklet/arrow velocity (3D) [au]
    t         ... tracklet/arrow reference epoch [JD/MJD]
    """
    
    # speed of light in au/day
    c_aupd = 173.145

    # Transform RADEC observations into positions on the unit sphere (US)
    xyz = tr.radec2icrfu(df['RA'], df['DEC'], deg=True)

    # Those are the line of sight (LOS) vectors
    los = np.array([xyz[0], xyz[1], xyz[2]]).T

    # Use the position of the observer and the LOS to project the position of
    # the asteroid onto a heliocentric great circle with radius r
    observer = df[['x_obs', 'y_obs', 'z_obs']].values

    # Calculate how much the heliocentric distance changes
    # during the obsevations based on assumed dr/dt
    dt = tref-df['time'].values
    dr = drdt*dt
    r_plus_dr = r+dr

    # Heliocentric postions of the observed asteroids
    posu = ls.sphere_line_intercept(los, observer, r_plus_dr)

    if(verbose):
        print('Heliocentric positions generated.')
 
    
    # tracklet position for filtered pairs
    x = posu[:-1,:]
    # tracklet time
    t = df['time'].values
    # tracklet velocity through forward differencing
    va = []
    vapp = va.append
    dt = t[1:]-t[0:-1]
    dx = posu[1:,:]-posu[0:-1,:]
    for d in range(0,3):
        vapp(np.divide(dx[:,d],dt))
    v = np.array(va).T
    t= df['time'].values[:-1]
    # correct arrows for light travel time
    if(lttc):
        if(verbose):
            print('(Linear correction for light travel time aberration...')
        xo = observer[:-1,:]
        dist = vec.norm(x-xo)
        xl = x.T-dist/c_aupd*v.T
        return xl.T, v, t

    else:
        return x, v, t

2) Extract epochs of observations and query JPL Horizons for the corresponding observer states

In [25]:
# def observer_states_from_horizons(epochs_of_observation, observer_location,tstart,tstop, ephemeris_dt='1h'):
#     """Query JPL Horizons via astroquery to get sun-observer state vectors.
    
#     Parameters:
#     -----------
#     observer_location  ... Horizons identifyer of observer location, e.g. 'I11'
#     tstart             ... start time for ephemeris in Horizons format, e.g. 'JD2456789.5'
#     tstop              ... end time for ephemeris in Horizons format, e.g. 'JD2456799.5'
#     ephemeris_dt       ... Time step for ephemeris query. 
#                            Typically 1h since the actual times will be interpolated later.
    
#     Returns:
#     --------
#     observer_xyz       ... Heliocentric observer positions at gridded epochs in [au].
#     observer_vxyz      ... Heliocentric observer velocities at gridded epochs in [au].
#     observer_jd        ... Gridded ephemeris epochs (JD / TDB)
    
#     External Function Requirements:
#     -------------------------------
#     # External API's
#     from astroquery.jplhorizons import Horizons
    

    
#     """
#     try:
#         # Get observer locations (caution: choose the right plane of reference and direction of the vectors!)
#         # check query by copy/pasting the output of print(observer_sun.uri) into a webbrowser if there are problems.
#         observer_sun=Horizons(id='Sun', location=observer_location, id_type='majorbody',
#                       epochs={'start':tstart, 'stop':tstop,
#                       'step':ephemeris_dt})

#         xyz=observer_sun.vectors()['x','y','z']
#         vxyz=(observer_sun.vectors())['vx','vy','vz']
#         jd=(observer_sun.vectors())['datetime_jd']
        
#         #We need the sun-observer vector not the observer-sun vector
#         observer_xyz=(-1)*np.array([xyz['x'],xyz['y'],xyz['z']])
#         observer_vxyz=(-1)*np.array([vxyz['vx'],vxyz['vy'],vxyz['vz']])
#         observer_jd=np.array(jd)
        
#     except:
#         print("Error in observer_state_from_horizons: potential online ephemeris query failure.")
#         raise
        
#     return observer_jd, observer_xyz, observer_vxyz



In [26]:
# def get_observer_states(observation_epochs, observer_location='I11', ephemeris_dt='1h'):
#     """Produce sun-observer state vectors at observation epochs.
    
#     Parameters:
#     -----------
#     observation_epochs         ... Numpy array of observation epochs [JD] 
#     observer_location          ... Horizons identifyer of observer location, e.g. 'I11'
#     ephemeris_dt               ... Time step for ephemeris query. 
#                                    Typically 1h since the actual times will be interpolated later.
    
#     Returns:
#     --------
#     observer_positions         ... Heliocentric observer positions at observation epochs in [au].
    
    
#     External Function Requirements:
#     -------------------------------

#     # Interpolation
#     import scipy.interpolate as spi
    
#     # time transform
#     mjd2jd                         ... change modified Julian date to Julian date, timescale TDB)
    
#     # NASA JPL HORIZONS API call wrapper
#     observer_states_from_horizons  ... Wrapper function for JPL Horizons state query via astropy
    
#     """

#     tmin = np.min(observation_epochs)
#     tmax = np.max(observation_epochs)
    
#     #Start and stop times of the survey
#     tstart = 'JD'+str(tmin-1.)
#     tstop = 'JD'+str(tmax+1.)

#     epochs = np.unique(observation_epochs)


#     [observer_jd,observer_xyz,observer_vxyz] = observer_states_from_horizons(observation_epochs,
#                                                                              observer_location,
#                                                                              tstart,tstop, ephemeris_dt)
        
#     # Interpolate heliocentric observer positions to the actual observation epochs
#     ir = spi.CubicHermiteSpline(observer_jd, observer_xyz,observer_vxyz, axis=1, extrapolate=None)
#     observer_positions = ir(obs_epochs).T
#     # Interpolate heliocentric observer velocities to the actual observation epochs
#     dirdt=ir.derivative(nu=1)
#     observer_velocities = dirdt(obs_epochs).T
    
    
#     return observer_positions, observer_velocities

In [27]:
#Observer location MPC code
#observer_location='I11'
observer_location='I11'
#Epochs of observation
obs_epochs=tr.mjd2jd(dfjpl['epoch_mjd'].values)
#time between observations
ephemeris_dt='1h'

In [28]:
[observer_xyz, observer_vxyz]=ephem.get_observer_states(obs_epochs,observer_location,ephemeris_dt)

In [29]:
observer_xyz

array([[-8.08596987e-01, -5.99238884e-01, -2.64921787e-05],
       [-8.08596987e-01, -5.99238884e-01, -2.64921787e-05],
       [-8.08596987e-01, -5.99238884e-01, -2.64921787e-05],
       ...,
       [-7.39834101e-01, -6.84768080e-01, -1.58123838e-06],
       [-7.39834101e-01, -6.84768080e-01, -1.58123838e-06],
       [-7.39834101e-01, -6.84768080e-01, -1.58123838e-06]])

In [30]:
observer_vxyz

array([[ 9.82560033e-03, -1.40536344e-02,  7.59332502e-05],
       [ 9.82560033e-03, -1.40536344e-02,  7.59332502e-05],
       [ 9.82560033e-03, -1.40536344e-02,  7.59332502e-05],
       ...,
       [ 1.15914466e-02, -1.25720671e-02, -5.22642083e-05],
       [ 1.15914466e-02, -1.25720671e-02, -5.22642083e-05],
       [ 1.15914466e-02, -1.25720671e-02, -5.22642083e-05]])

In [31]:
dfobs=dfjpl[['object_name','epoch_mjd','ra_deg','dec_deg']].rename(columns={'object_name':'obj','epoch_mjd':'time','ra_deg':'RA','dec_deg':'DEC'})
#epoch_mjd	ra_deg	ra_sigma_deg	dec_deg	dec_sigma_deg	mag	mag_sigma	filter_id	field_id	nn	object_name

In [32]:
dfobs['x_obs']=observer_xyz[:,0]
dfobs['y_obs']=observer_xyz[:,1]
dfobs['z_obs']=observer_xyz[:,2]
dfobs['vx_obs']=observer_vxyz[:,0]
dfobs['vy_obs']=observer_vxyz[:,1]
dfobs['vz_obs']=observer_vxyz[:,2]

In [33]:
dfobs['night']=ls.lsstNight(dfobs['time'],dfobs['time'].min()).astype('Int32')

In [34]:
dfobs

,obj,time,RA,DEC,x_obs,y_obs,z_obs,vx_obs,vy_obs,vz_obs,night
0,S1009GjOa,52391.002282,171.368899,-12.504937,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
1,S1001QUsa,52391.002282,169.318742,-13.021633,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
2,S10036B8a,52391.002282,170.375067,-12.218251,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
3,S100aAWQa,52391.002282,171.392970,-14.233830,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
4,S1001DTsa,52391.002282,171.308411,-14.222651,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
...,...,...,...,...,...,...,...,...,...,...,...
15458291,FD,52397.429729,339.306787,-6.740917,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15458292,FD,52397.429729,336.475206,-5.178729,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15458293,FD,52397.429729,337.601773,-7.451194,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15458294,FD,52397.429729,338.029519,-6.705724,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6


In [35]:
dffalse=dfobs[dfobs['obj']=='FD']

In [36]:
dffalse

,obj,time,RA,DEC,x_obs,y_obs,z_obs,vx_obs,vy_obs,vz_obs,night
772,FD,52391.002282,169.809327,-12.296747,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
773,FD,52391.002282,169.572564,-11.974931,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
774,FD,52391.002282,171.304416,-14.200303,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
775,FD,52391.002282,169.397425,-13.948415,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
776,FD,52391.002282,170.042972,-13.016745,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
...,...,...,...,...,...,...,...,...,...,...,...
15458291,FD,52397.429729,339.306787,-6.740917,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15458292,FD,52397.429729,336.475206,-5.178729,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15458293,FD,52397.429729,337.601773,-7.451194,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15458294,FD,52397.429729,338.029519,-6.705724,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6


In [37]:
dfnoise=dfobs[dfobs['obj']=='NS']

In [38]:
dfobs2=dfobs[dfobs['obj']!='FD']

In [39]:
dfobs3=dfobs2[dfobs2['obj']!='NS']

In [40]:
dfobs3

,obj,time,RA,DEC,x_obs,y_obs,z_obs,vx_obs,vy_obs,vz_obs,night
0,S1009GjOa,52391.002282,171.368899,-12.504937,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
1,S1001QUsa,52391.002282,169.318742,-13.021633,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
2,S10036B8a,52391.002282,170.375067,-12.218251,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
3,S100aAWQa,52391.002282,171.392970,-14.233830,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
4,S1001DTsa,52391.002282,171.308411,-14.222651,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
...,...,...,...,...,...,...,...,...,...,...,...
15455740,S100bGBsa,52397.429729,338.946285,-5.232156,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15455741,S100noswa,52397.429729,337.373092,-7.302586,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15455742,S1008g5ma,52397.429729,338.461937,-5.578753,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15455743,S1007Btva,52397.429729,338.976612,-5.941488,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6


In [41]:
dfobs3[dfobs3['obj'].isin((dfobs3.groupby('obj').count()>6).index)]

,obj,time,RA,DEC,x_obs,y_obs,z_obs,vx_obs,vy_obs,vz_obs,night
0,S1009GjOa,52391.002282,171.368899,-12.504937,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
1,S1001QUsa,52391.002282,169.318742,-13.021633,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
2,S10036B8a,52391.002282,170.375067,-12.218251,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
3,S100aAWQa,52391.002282,171.392970,-14.233830,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
4,S1001DTsa,52391.002282,171.308411,-14.222651,-0.808597,-0.599239,-0.000026,0.009826,-0.014054,0.000076,0
...,...,...,...,...,...,...,...,...,...,...,...
15455740,S100bGBsa,52397.429729,338.946285,-5.232156,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15455741,S100noswa,52397.429729,337.373092,-7.302586,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15455742,S1008g5ma,52397.429729,338.461937,-5.578753,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6
15455743,S1007Btva,52397.429729,338.976612,-5.941488,-0.739834,-0.684768,-0.000002,0.011591,-0.012572,-0.000052,6


In [42]:
value_counts = dfobs3["obj"].value_counts()

In [43]:
value_counts.index[0:100]

Index(['S1005eJDa', 'S1005kh2a', 'S100j1hLa', 'S100f2A7a', 'S1003q74a',
       'S1007PWTa', 'S1003xSRa', 'S1003Pkba', 'S1006aiKa', 'S10078wha',
       'S1005NEpa', 'S100eFFOa', 'S1001kyBa', 'S10049Bxa', 'S1002rvXa',
       'S1002ddya', 'S1003GfCa', 'S1007Fs6a', 'S1000mJ7a', 'S1003pQLa',
       'S1007f6ha', 'S1001Nana', 'S100jJFEa', 'S10046DWa', 'S10027UDa',
       'S10068Z0a', 'S100iTaza', 'S100cut0a', 'S100hysEa', 'S0180604a',
       'S1003QLwa', 'S100uJ0ta', 'S1002bena', 'S1003ixDa', 'S10011Fba',
       'S10001nIa', 'S1003wHDa', 'S1002U83a', 'S100coiIa', 'S100bTuAa',
       'S1001ayCa', 'S1007xT8a', 'S1003klXa', 'S1000diga', 'S1002bnwa',
       'S100iuJ2a', 'S10054NQa', 'S1007LNYa', 'S100b2CGa', 'S101bhLJa',
       'S1006ROqa', 'S10003uea', 'S100bcNDa', 'S1001GMZa', 'S10079TRa',
       'S100fxvTa', 'S1001m4ba', 'S1000epJa', 'S1003sIka', 'S1000drja',
       'S1005gVwa', 'S1000om9a', 'S1003Qqma', 'S100dLvXa', 'S1005Ee6a',
       'S1000cqGa', 'S100bbxAa', 'S100lwrua', 'S1000itga', 'S100

In [44]:
dfobs4 = dfobs3[(dfobs3['obj'].isin(value_counts.index[value_counts.values >= 10].values))]

In [45]:
dfobs4.groupby('obj').count()

,time,RA,DEC,x_obs,y_obs,z_obs,vx_obs,vy_obs,vz_obs,night
obj,,,,,,,,,,
S0000101a,12,12,12,12,12,12,12,12,12,12
S0016009a,11,11,11,11,11,11,11,11,11,11
S0016229a,11,11,11,11,11,11,11,11,11,11
S0018147a,10,10,10,10,10,10,10,10,10,10
S0020834a,11,11,11,11,11,11,11,11,11,11
...,...,...,...,...,...,...,...,...,...,...
S103g0v1a,10,10,10,10,10,10,10,10,10,10
S103gOCCa,15,15,15,15,15,15,15,15,15,15
S103kBvWa,12,12,12,12,12,12,12,12,12,12


In [46]:
#dfobs5=dfobs4[dfobs4['obj'].isin(['S0000101a','S0000786a','S0001048a','S103gOCCa'])]
#dfobs5=dfobs4[dfobs4['obj'].isin(['S103gOCCa'])]
dfobs5=dfobs4[dfobs4['obj'].isin(['S0000101a'])]
#dfobs5=dfobs3[(dfobs3['obj'].isin(value_counts.index[0:20]))]

In [47]:
dfobs5

,obj,time,RA,DEC,x_obs,y_obs,z_obs,vx_obs,vy_obs,vz_obs,night
297496,S0000101a,52391.157220,236.453148,-39.726299,-0.807055,-0.601419,-0.000013,0.010078,-0.014053,0.000087,0
299979,S0000101a,52391.157668,236.453078,-39.726354,-0.807051,-0.601425,-0.000013,0.010078,-0.014052,0.000087,0
336673,S0000101a,52391.165514,236.451099,-39.727252,-0.806972,-0.601535,-0.000012,0.010091,-0.014047,0.000085,0
339389,S0000101a,52391.165962,236.451025,-39.727254,-0.806967,-0.601542,-0.000012,0.010092,-0.014047,0.000085,0
722651,S0000101a,52391.271123,236.424720,-39.738974,-0.805898,-0.603013,-0.000005,0.010227,-0.013935,0.000044,0
737592,S0000101a,52391.273894,236.424015,-39.739285,-0.805870,-0.603052,-0.000005,0.010230,-0.013931,0.000043,0
740257,S0000101a,52391.274343,236.423928,-39.739339,-0.805865,-0.603058,-0.000005,0.010230,-0.013930,0.000043,0
10660542,S0000101a,52396.228143,235.120786,-40.241723,-0.753421,-0.669373,-0.000003,0.011321,-0.013061,0.000059,5
10826902,S0000101a,52396.248490,235.114651,-40.243606,-0.753191,-0.669639,-0.000002,0.011343,-0.013035,0.000049,5
13134039,S0000101a,52397.107037,234.865552,-40.321019,-0.743559,-0.680639,-0.000012,0.011365,-0.012968,0.000092,6


In [48]:
#dfobs=pd.concat([dfobs5,dffalse,dfnoise])
#dfobs=dfobs4[:200000]

In [49]:
dfobs=dfobs5

In [50]:
dfobs

,obj,time,RA,DEC,x_obs,y_obs,z_obs,vx_obs,vy_obs,vz_obs,night
297496,S0000101a,52391.157220,236.453148,-39.726299,-0.807055,-0.601419,-0.000013,0.010078,-0.014053,0.000087,0
299979,S0000101a,52391.157668,236.453078,-39.726354,-0.807051,-0.601425,-0.000013,0.010078,-0.014052,0.000087,0
336673,S0000101a,52391.165514,236.451099,-39.727252,-0.806972,-0.601535,-0.000012,0.010091,-0.014047,0.000085,0
339389,S0000101a,52391.165962,236.451025,-39.727254,-0.806967,-0.601542,-0.000012,0.010092,-0.014047,0.000085,0
722651,S0000101a,52391.271123,236.424720,-39.738974,-0.805898,-0.603013,-0.000005,0.010227,-0.013935,0.000044,0
737592,S0000101a,52391.273894,236.424015,-39.739285,-0.805870,-0.603052,-0.000005,0.010230,-0.013931,0.000043,0
740257,S0000101a,52391.274343,236.423928,-39.739339,-0.805865,-0.603058,-0.000005,0.010230,-0.013930,0.000043,0
10660542,S0000101a,52396.228143,235.120786,-40.241723,-0.753421,-0.669373,-0.000003,0.011321,-0.013061,0.000059,5
10826902,S0000101a,52396.248490,235.114651,-40.243606,-0.753191,-0.669639,-0.000002,0.011343,-0.013035,0.000049,5
13134039,S0000101a,52397.107037,234.865552,-40.321019,-0.743559,-0.680639,-0.000012,0.011365,-0.012968,0.000092,6


In [51]:
# Convert observations and observer states to pandas dataframe
# obsa=np.array(obs)
# dfradec=pd.DataFrame(np.hstack(obsa).T,columns=['objId','time','RA','DEC'])
# dfradec['objId']=dfradec['objId'].astype('int')
# dfxyz=pd.DataFrame(obsxyz.data,columns=['x','y','z'])*(-1)
# dfxyz2=dfxyz
# for i in range(len(asteroids)):
#        dfxyz2=pd.concat([dfxyz2,dfxyz])

In [52]:
# dfxyz2.reset_index(drop=True);

In [53]:
# dfobs=(dfradec.join(dfxyz2.reset_index(drop=True))).rename(columns={'x':'x_obs','y':'y_obs','z':'z_obs'})

## If LSST observations are used from file uncomment the following cell:

In [54]:
#dfobs=pd.read_csv('../data/lsst_sso_534_obs.tar.gz',sep='\s+',nrows=1200)

In [55]:
# check observation dataframe: must contain time, RA, DEC, x_obs, y_obs, z_obs
dfobs[0:10]

,obj,time,RA,DEC,x_obs,y_obs,z_obs,vx_obs,vy_obs,vz_obs,night
297496,S0000101a,52391.157220,236.453148,-39.726299,-0.807055,-0.601419,-0.000013,0.010078,-0.014053,0.000087,0
299979,S0000101a,52391.157668,236.453078,-39.726354,-0.807051,-0.601425,-0.000013,0.010078,-0.014052,0.000087,0
336673,S0000101a,52391.165514,236.451099,-39.727252,-0.806972,-0.601535,-0.000012,0.010091,-0.014047,0.000085,0
339389,S0000101a,52391.165962,236.451025,-39.727254,-0.806967,-0.601542,-0.000012,0.010092,-0.014047,0.000085,0
722651,S0000101a,52391.271123,236.424720,-39.738974,-0.805898,-0.603013,-0.000005,0.010227,-0.013935,0.000044,0
737592,S0000101a,52391.273894,236.424015,-39.739285,-0.805870,-0.603052,-0.000005,0.010230,-0.013931,0.000043,0
740257,S0000101a,52391.274343,236.423928,-39.739339,-0.805865,-0.603058,-0.000005,0.010230,-0.013930,0.000043,0
10660542,S0000101a,52396.228143,235.120786,-40.241723,-0.753421,-0.669373,-0.000003,0.011321,-0.013061,0.000059,5
10826902,S0000101a,52396.248490,235.114651,-40.243606,-0.753191,-0.669639,-0.000002,0.011343,-0.013035,0.000049,5
13134039,S0000101a,52397.107037,234.865552,-40.321019,-0.743559,-0.680639,-0.000012,0.011365,-0.012968,0.000092,6


In [56]:
dfobs.keys()
# should be Index(['obj', 'time', 'RA', 'DEC', 'x_obs', 'y_obs', 'z_obs'], dtype='object')

Index(['obj', 'time', 'RA', 'DEC', 'x_obs', 'y_obs', 'z_obs', 'vx_obs',
       'vy_obs', 'vz_obs', 'night'],
      dtype='object')

In [57]:
dfobs['obsId']=dfobs.index

/epyc/projects/pytrax_play/envs/pytrax_s/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [58]:
df1=dfobs['obj'].unique()


In [59]:
#Making numerical categories from object data
dfobs['objId']=dfobs.obj.astype('category').cat.codes

/epyc/projects/pytrax_play/envs/pytrax_s/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [60]:
#save a csv output 
#df.to_csv('observations_database.csv')

In [61]:
dfobs[['x_obs','y_obs','z_obs']].values

array([[-8.07055451e-01, -6.01418794e-01, -1.27587197e-05],
       [-8.07050936e-01, -6.01425089e-01, -1.27197040e-05],
       [-8.06971811e-01, -6.01535324e-01, -1.20433787e-05],
       [-8.06967290e-01, -6.01541617e-01, -1.20051782e-05],
       [-8.05898310e-01, -6.03013484e-01, -4.92865734e-06],
       [-8.05869967e-01, -6.03052091e-01, -4.80744317e-06],
       [-8.05865373e-01, -6.03058346e-01, -4.78817408e-06],
       [-7.53421453e-01, -6.69373188e-01, -2.93614700e-06],
       [-7.53190872e-01, -6.69638681e-01, -1.83821461e-06],
       [-7.43559374e-01, -6.80638804e-01, -1.16733680e-05],
       [-7.43554293e-01, -6.80644601e-01, -1.16322385e-05],
       [-7.43058556e-01, -6.81207967e-01, -7.76294924e-06]])

In [62]:
threeobs=dfobs.iloc[[1,8,10]]

In [63]:
# help(iod.gauss)

In [64]:
# coords_eq_ang=threeobs[['RA','DEC']].values
# t=threeobs['time'].values
# coords_obs=threeobs[['x_obs','y_obs','z_obs']].values

In [65]:
# gauss_sol=iod.gaussIOD(coords_eq_ang, t, coords_obs, velocity_method='gibbs', light_time=True, iterate=True, mu=0.0002959122082855911, max_iter=10, tol=1e-15)

In [66]:
# gauss_sol

In [67]:
# oe_sol=[]
# for sol in gauss_sol:
#     print(sol)
#     oe_sol.append(tr.Cartesian2Cometary(sol[0], sol[1:], mu=0.01720209895**2))

In [68]:
# oe_sol

In [69]:
"S0000101a,1.243770845037,0.530662969779,11.9149689089,340.42111262526,357.558228330142,52582.76968203646,18.0507717,52389"

'S0000101a,1.243770845037,0.530662969779,11.9149689089,340.42111262526,357.558228330142,52582.76968203646,18.0507717,52389'

In [70]:
S101a_cart=tr.cometary2cartesian(52389, np.array([1.243770845037,0.530662969779,11.9149689089,340.42111262526,357.558228330142,52582.76968203646]),frame='ecliptic')

S101a=st.State(S101a_cart,52389)
S101a.frame='ecliptic'

In [71]:
S101a.x

array([-1.3939691 , -1.70894225, -0.43831377])

In [72]:
# def radecResiduals(df, epoch, state_asteroid, output_units='deg', **kwargs):
#     """Calculate O-C values in Right Ascension (RA) and Declination (DEC) for a given asteroid state. 
#     The state is propagated to all observation epochs and the corresponding RA and DEC values are compared
#     to the corresponding observations. Heliocentric ecliptic states for the observer are required for
#     every obserbation epoch. 
    
#     Parameters:
#     -----------
#     df                ... Pandas DataFrame containing nightly RA and DEC [deg], time [JD, MJD] UTC,
#                           heliocentric ecliptic observer positions and velocities [au]
                
#     epoch             ... epoch of asteroid state [JD, MJD], timescale: TDB
#     state_asteroid    ... heliocentric ecliptic state of asteroid at epoch (x,y,z,vx,vy,vz), [au, au/day]
    
#     Keyword arguments:
#     ------------------
#     output_units      ... units for O-C results: 'deg', 'arcsec', 'rad'
#     epoch_timescale   ... time scale for observation epoch ('utc', 'tdb'), default utc
#     state_timescale   ... time scale for asteroid and observer state epoch ('utc', 'tdb'), default tdb
#     deg               ... True (default): angles in degrees, False: angles in radians
#     lttc              ... True (default): correct for light travel time \
#                           (needs entire state including velocities) 
#     propagator        ... propagation algorithm for asteroid state propagation: 
#                           '2body' (default), 'linear', 'nbody'                      
#     Returns:
#     --------
#     rms               ... root mean square (RMS) of RA and DEC O-Cs [arcseconds]
#     dra               ... Right Ascension O-C values for all observations [arcseconds]
#     ddec              ... Declination O-C values for all observations [arcseconds]
#     """
#     dt = epoch-df['time'].values

#     ephemeris = []
#     ephemeris_app = ephemeris.append
#     nobs = len(dt)
#     for i in range(nobs):
#         state_observer = df[['x_obs', 'y_obs', 'z_obs','vx_obs', 'vy_obs', 'vz_obs']].values[i]
    
#     # propagate orbit to all observation time
#         pstate = prop.propagateState(state_asteroid[0:3],state_asteroid[3:6], 
#                                      epoch, df['time'].values[i], propagator='2body')
#         state_asteroid_prop = np.array(pstate[0:2]).flatten()
#         ephemeris_app(tr.state2ephemeris(epoch, state_asteroid_prop, state_observer, frame='ecliptic', lttc=True, timescale_state='tdb',
#                                         timescale_epoch='utc')) 
    
#     # O-C
#     ephemeris_array = np.array(ephemeris)
    
# #     print('observed', np.array([df['RA'].values, df['DEC'].values]).T)
# #     print('calculated', np.array(ephemeris)[:,0:2])
    
#     dra = df['RA'].values - ephemeris_array[:,0]
#     ddec =  df['DEC'].values - ephemeris_array[:,1]
    
#     dradec = np.array([dra, ddec]).flatten()
    
#     rms = np.sqrt(np.dot(dradec.T, dradec)/nobs)

#     if(output_units == 'deg'):
#         return rms, dra, ddec
#     elif(output_units == 'rad'):
#         return np.deg2rad(rms), np.deg2rad(dra), np.deg2rad(ddec)
#     elif(output_units == 'arcsec'):
#         return rms*3600, dra*3600, ddec*3600
#     else:
#         raise Exception('Error in radecResiduals: unknown output unit.')
    
  
    

In [73]:
import ephemeris as ep

In [74]:
[rms, dra, ddec] = ep.radecResiduals(dfobs, 52389, S101a_cart, output_units='arcsec')

In [75]:
#gauss_sol[0][0]

NameError: name 'gauss_sol' is not defined

In [76]:
dfobs['time'].values

array([52391.15722 , 52391.157668, 52391.165514, 52391.165962,
       52391.271123, 52391.273894, 52391.274343, 52396.228143,
       52396.24849 , 52397.107037, 52397.107484, 52397.150965])

In [77]:
#gauss_sol[1][1:7]

NameError: name 'gauss_sol' is not defined

In [78]:
#[rms, dra, ddec] = radec_residuals(dfobs, gauss_sol[0][0], gauss_sol[0][1:7])

NameError: name 'radec_residuals' is not defined

In [79]:
#tst_cart=tr.cometary2cartesian(52389, np.array([1.,0.,1,0,0,52582]))

#[rms, dra, ddec] = radecResiduals(dfobs, 52389, tst_cart)

In [80]:
print(rms)

24.62991695089818


In [81]:
print(dra)

[-24.16395438 -24.01766392 -24.15669209 -24.02375114 -24.17550844
 -24.20702846 -24.11395852 -25.16364826 -25.15258296 -25.19581544
 -25.20218556 -25.25563447]


In [82]:
print(ddec)

[1.09150207 1.08040147 1.06718858 1.24179513 1.35189269 1.32314862
 1.30531342 2.06643241 2.07894485 2.12339405 2.12066823 2.15315691]


In [ ]:
#[err, jdSel, raSel, deSel, stnPosSel] = iod.sel3obs(dfobs.time.values, dfobs.RA.values, dfobs.DEC.values, dfobs[['x_obs','y_obs','z_obs']].values)

In [ ]:
#stnPosSel

In [ ]:
def omc(epoch_obs, observations, epoch_orb, orbit_state, timescale_obs, timescale_orbit, *args, **kwargs):
    
    # propagate orbit to observation times
    times_obs = Time(epoch_obs,scale=timescale_obs)
    times_prop = Time(times_obs, scale=timescale_orbit)
    
    for t in times_prop:
        propagate
        
        
    
    tr.state2ephemeris(times, state_asteroid, state_observer, **kwargs)

In [ ]:
#jdSel

In [ ]:
#iod.gaussPoly(jdSel, raSel, deSel, stnPosSel, True)

In [ ]:
# [err, carSoln, epoch] = iod.gaussGrid(jdSel, raSel, deSel, stnPosSel)

In [ ]:
#carSoln

In [ ]:
# # --------------------------------
# # Loop on solutions and print them on screen
# nSol = len(carSoln)
# print '==========================================='
# print 'The method of Gauss found %d solutions' % nSol
# for i in range(nSol):

#     # Here there could be some logic to discard the solutions that
#     # should not be passed to differential corrections, e.g.:
#     # - discard the solutions that did not converge
#     # - discard solution with too large eccentricity

#     # Compute orbital elements
#     [err, elem] = car2com(carSoln[i,:], epoch[i])
#     if err != 0:
#         print 'car2com exited with error code %d' % err
#         sys.exit(7)

#     # Skip large eccentricities
#     if elem[0] > 100.0:
#         print 'Solution %d has a large ecc=%s, skipping' % (i, elem[0])
#         continue

#     # Compute residuals with two body motion
#     [err, raRes, deRes, rms] = \
#       compKepRes(jd, ra, de, stnPos, carSoln[i,:], epoch[i])
#     if err != 0:
#         print 'compKepRes exited with error code %d' % err
#         sys.exit(8)

#     # Print solution on screen
#     print 'EPOCH=%s EC=%.9f QR=%.9f TP=%.6f OM=%.7f W=%.7f IN=%.7f' % \
#       (epoch[i], elem[0], elem[1], elem[2], elem[3], elem[4], elem[5])
#     print 'The prefit RMS on the full astrometric dataset is %.3f arcsec' % rms
#     if mode == 'poly' and conv[i]:
#         print 'Solution converged with gaussIter'
#     if mode == 'poly' and not conv[i]:
#         print 'Solution not converged with gaussIter'

### Let's build our KDTree and create arrows 


Run a quick test to see if the arrow generation was successul:

In [ ]:
df_grouped_by_night=dfobs.groupby('night')

In [ ]:
len(df_grouped_by_night.groups)

In [ ]:
#r=2.6
#drdt=-0.0001
cr=0.01
#cr=0.005/180*np.pi*2
# max temporal separation for tracklet observations (days)
ct_max=8/24
# min temporal separation for tracklet observations, e.g. exposure time (days)
ct_min=30/86400 


#use astrometric uncertainty to determine clustering error (150mas (3sigma) )
#astrometric_uncertainty=300/1000/3600/180*np.pi

# DBSCAN clustering parameters
#eps=astrometric_uncertainty*r

eps=0.01
min_samples=3

In [ ]:
def heliolinc2(dfobs,r,drdt,cr,ct_min,ct_max,clustering_algorithm='dbscan',lttc=False,verbose=True):
    """HelioLinC2 (Heliocentric Linking in Cartesian Coordinates) algorithm

    Parameters:
    -----------
    dfobs ... Pandas DataFrame containing object ID (objId), observation ID (obsId),
               time, night, RA [deg], DEC[deg], observer x,y,z [au]


    r      ... assumed heliocentric distance [au]
    drdt   ... dr/dt assumed heliocentric radial velocity [au/day]
    cr     ... clustering radius [au]
    ct_min ... minimum timespan between observations to allow for trackelt making [days]
    ct_max ... maximum timespan between observations to allow for tracklet making [days]
    clustering_algorithm ... clustering_algorithm (currently either 'dbscan' or 'kdtree')
    lttc   ... Light travel time correction
    verbose... Print progress statements


    Returns:
    --------
    obs_in_cluster_df ... Pandas DataFrame containing linked observation clusters (no prereduction)
    """

#     xpall=[]
#     vpall=[]
    xar=[]
    var=[]
    tar=[]
    obsids_night=[]

    # the following two arrays are for testing purposes only
    objid_night=[]
    tobs_night=[]

    for n in df_grouped_by_night.groups.keys():
        if (verbose):
            print('Processing night ',n)
        # SELECT NIGHT FROM OBSERVATIONS DATA BASE
        idx=df_grouped_by_night.groups[n].values
        df=dfobs.loc[idx,:].reset_index(drop=True)
        #tref=(df['time'].max()+df['time'].min())*0.5
        tref=(dfobs['time'].max()+dfobs['time'].min())*0.5

        # GENERATE ARROWS / TRACKLETS FOR THIS NIGHT
        [xarrow_night, 
         varrow_night, 
         tarrow_night, 
         goodpairs_night]=ls.MakeHeliocentricArrows(df,r,drdt,tref,cr,ct_min,
                                                        ct_max,v_max=1,lttc=False,
                                                        filtering=True,verbose=True,eps=cr)
        # ADD TO PREVIOUS ARROWS
        if (len(xarrow_night)<1):
            if (verbose):
                print('no data in night ',n)
        else:
            xar.append(xarrow_night)
            var.append(varrow_night)
            tar.append(tarrow_night)
            obsids_night.append(df['obsId'].values[goodpairs_night])
            objid_night.append(df['objId'].values[goodpairs_night])
            tobs_night.append(df['time'].values[goodpairs_night])


    if (len(xar)<1):
        if (verbose):
            print('No arrows for the current r, dr/dt pair. ',n)
    else:    
        xarrow=np.vstack(xar)
        varrow=np.vstack(var)
        tarrow=np.hstack(tar)
        obsids=np.vstack(obsids_night)

    # the following two arrays are for testing purposes only
        objids=np.vstack(objid_night)
        tobs=np.vstack(tobs_night)

    # PROPAGATE ARROWS TO COMMON EPOCH
        if (verbose):
            print('Propagating arrows...')
        tprop=(dfobs['time'].max()+dfobs['time'].min())*0.5
    #tprop=dfobs['time'].max()+180
        [xp,vp,dt] = ls.PropagateArrows(xarrow,varrow,tarrow,tprop,propagator='2body')
    #[xp,vp,dt] = ls.propagate_arrows_2body(xarrow,varrow,tarrow,dfobs['time'].max()+360)

        rnorm=(r/ls.norm(vp))
        vpn=vp*np.array([rnorm,rnorm,rnorm]).T
        xpvp=np.hstack([xp,vpn])

#       # CLUSTER WITH DBSCAN

        if (verbose):
            print('Clustering arrows...')
#       # CLUSTER PROPAGATED STATES (HERE IN REAL SPACE, BUT COULD BE PHASE SPACE)               
        if(clustering_algorithm=='dbscan'):
            db=cluster.DBSCAN(eps=eps,min_samples=min_samples,n_jobs=4).fit(xp)

#       # CONVERT CLUSTER INDICES TO OBSERVATION INDICES IN EACH CLUSTER
            try:
                obs_in_cluster, labels = observations_in_cluster(dfobs,obsids,db,garbage=False)
                obs_in_cluster_df=pd.DataFrame(zip(labels,obs_in_cluster),columns=['clusterId','obsId'])
            except: 
                print('Error in constructing cluster dataframe.')

        elif (clustering_algorithm=='kdtree'):
    # CLUSTER WITH KDTree
            if (verbose):
                print('Clustering arrows...')
            tree = scsp.KDTree(xp)
            db = tree.query(xp, k=8, p=2, distance_upper_bound=eps)

            if (verbose):
                print('Deduplicating observations in clusters...')
            obs = []
            obs_app = obs.append
            arrow_idx = np.array(db,dtype="int64")[1]
            nan_idx = arrow_idx.shape[0]
            for i in arrow_idx:
                entries = i[i<nan_idx]
                if(len(entries)) > 1:
                     obs_app([np.unique(np.ravel(obsids[entries]))])

            obs_in_cluster_df = pd.DataFrame(obs,columns=['obsId'])
            obs_in_cluster_df['clusterId']=obs_in_cluster_df.index.values
            obs_in_cluster_df=obs_in_cluster_df[['clusterId','obsId']]

        else:
            raise ('Error in heliolinc2: no valid clustering algorithm selected') 

    #COMMON CODE
        obs_in_cluster_df['r'] = r
        obs_in_cluster_df['drdt'] = drdt
        obs_in_cluster_df['cluster_epoch'] = tprop
        #xpall.append(xp)
        #vpall.append(vp)

        return obs_in_cluster_df

In [ ]:
cr

In [ ]:
df_grouped_by_night.groups.keys()

In [ ]:
# %%time
# [xarrow, varrow, tarrow, goodpairs]=ls.create_heliocentric_arrows(df,r,drdt,tref,cr,ct,lttc=False,filtering=True,verbose=True,eps=cr/3)

In [ ]:
# %%time
# from joblib import Parallel, delayed 

# # range of heliocentric radii
# rall=np.arange(1.15,3,0.1)
# #rall=[2.5]
# # range of heliocentric radial velocities
# drdtall=np.arange(-0.41,0.41,0.1)
# #drdtall=[-0.0001]

# clusters_df=[]

# def ProcessInput(drdt):
#     print('dr/dt=',drdt)
#     obs_in_cluster_df=heliolinc2(dfobs,r,drdt,cr,ct_min,ct_max,clustering_algorithm='dbscan',lttc=False,verbose=True)
#     return obs_in_cluster_df    

# for r in rall:
#     print('r=',r)
#     obs_in_cluster_df_p=Parallel(n_jobs=4)(delayed(ProcessInput) (drdt) for drdt in drdtall)
#     #if not obs_in_cluster_df_p.empty:
#     clusters_df.extend(obs_in_cluster_df_p)   

In [ ]:
# %%time
# from joblib import Parallel, delayed 

# # range of heliocentric radii
# #rall=np.arange(1.15,3,0.1)
# rall=np.arange(1.7,3,0.1)
# #rall=[2.5]
# # range of heliocentric radial velocities
# #drdtall=np.arange(-0.41,0.41,0.1)
# drdtall=np.arange(-0.11,0.11,0.02)
# #drdtall=[-0.0001]

# clusters_df=[]

# def ProcessInput(r):
#     #print('dr/dt=',drdt)
#     obs_in_cluster_df=heliolinc2(dfobs,r,drdt,cr,ct_min,ct_max,clustering_algorithm='dbscan',lttc=False,verbose=False)
#     return obs_in_cluster_df    

# with Parallel(n_jobs=6) as parallel:
#     for drdt in drdtall:
#         print('dr/dt=',drdt)
#         obs_in_cluster_df_p=parallel(delayed(ProcessInput) (r) for r in rall)
#         #if not obs_in_cluster_df_p.empty:
#         clusters_df.extend(obs_in_cluster_df_p)
#         print('total number of dataframes:',len(clusters_df))


In [ ]:
%%time
# range of heliocentric radii
#rall=np.arange(1.15,3,0.1)
rall=np.arange(1.05,3,0.1)
#rall=[2.5]
# range of heliocentric radial velocities
#drdtall=np.arange(-0.41,0.41,0.1)
drdtall=np.arange(-0.11,0.11,0.05)
#drdtall=[-0.0001]

clusters_df=[]

for r in rall:
    print('r=',r)
    for drdt in drdtall:
        print('dr/dt=',drdt)
        obs_in_cluster_df=heliolinc2(dfobs,r,drdt,cr,ct_min,ct_max,clustering_algorithm='dbscan', lttc=False, verbose=False)
        if not obs_in_cluster_df.empty:
            clusters_df.append(obs_in_cluster_df)
        print('total number of dataframes:',len(clusters_df))

In [ ]:
clusters_df

In [ ]:
# # GENERATE FINAL DATAFRAME WITH CLUSTERING RESULTS  
print('final step')
clustered_observations = (pd.concat(clusters_df)).reset_index(drop=True)  

In [ ]:
# DROP DUPLICATE CLUSTERS
clustered_observations_final=(clustered_observations.iloc[clustered_observations.astype(str).drop_duplicates(subset='obsId', keep="first").index]).reset_index(drop=True)    

In [ ]:
clustered_observations_final

In [ ]:
dfobs

In [ ]:
cof2['obsId'].values

In [ ]:
#clustered_observations.iloc[clustered_observations.astype(str).drop_duplicates(subset='obsId', keep="first").index].count()

In [ ]:
def radec_residuals(time,ra_o,dec_o,xyz_hel_observer,orbit_state,epoch):
    
    dt = time-epoch
    print(time,epoch)
    print('dt',dt)
    # Gaussian Gravitational Constant [au^1.5/Msun^0.5/D]
    gaussk = 0.01720209894846
    # default gravitational parameter [Gaussian units]
    gm = gaussk*gaussk
    
    xyz_hel_ast=[]
    xyz_hel_ast_app=xyz_hel_ast.append
    for i in range(len(dt)):
        xyz_hel_ast_app(sp.prop2b(gm, orbit_state, dt[i])[0:3])
    
    #xyz_hel_ast = state[0:3]
    #propagate orbit to all observation time
    [ra_c,dec_c] = IcrfHel2RaDecTopo_deg(xyz_hel_ast,xyz_hel_observer)
    
    dra = ra_o-ra_c
    ddec = dec_o-dec_c
    dradec=np.array(dra,ddec).T
    
    rms = np.sqrt(np.dot(dradec,dradec)/len(dra))
    print(dra)
    print(dradec)
    
    return rms,dra,ddec
    

In [ ]:
obs_idx=[]
obs_idx_ext=obs_idx.extend
cof2=clustered_observations_final

for index, row in cof2.iterrows():
    print(row.r,row.drdt)
    df=dfobs[dfobs['obsId'].isin(row['obsId'])]
    #print(df)
    xyz=obs2heliocentric_xyz(df,r,drdt,row['cluster_epoch'])
    dx=(xyz[1:]-xyz[0:-1])
    dt=(df['time'].values[1:]-df['time'].values[0:-1])
    
    va=[]
    vapp=va.append
    for d in range(0,3):
        vapp(np.divide(dx[:,d],dt))
    v = np.array(va).T
    
#    print(xyz[0])
#    print(v[0])
    asteroid_state=np.array([xyz[0],v[0]]).flatten()
    print(asteroid_state)
    [rms,dra,ddec]=radec_residuals(df['time'].values,
                                   df['RA'].values,
                                   df['DEC'].values,
                                   df[['x_obs', 'y_obs', 'z_obs']].values,
                                   asteroid_state,
                                   df['time'].values[0])
    print(rms)
    
    #print(row.obsId)
    #print(v)
    #interpolant=spi.interp1d(df['time'],xyz,kind='quadratic')
    #print(row['cluster_epoch'],interpolant(row['cluster_epoch']))

In [ ]:
help(sp.prop2b)

In [ ]:
len(clustered_observations)

In [ ]:
len(clustered_observations_final)

In [ ]:
#set_diff_df = pd.concat([clustered_observations.astype(str), clustered_observations_final.astype(str), clustered_observations_final.astype(str)]).drop_duplicates(keep=False)

In [ ]:
#set_diff_df

# Try IOD with THOR


In [ ]:
import thor

In [ ]:
#dup_idx=np.where(clustered_observations.astype(str).duplicated(subset='obsId',keep='first'))[0]

In [ ]:
#clustered_observations.iloc[dup_idx]

In [ ]:
clustered_observations_final

In [ ]:
def collapse_clusters(cdf):
       #for index, row in clusters_df.iterrows():
    vals=cdf.obsId.values
    subset_clusters=[]
    subset_clusters_app=subset_clusters.append
    subset_cluster_ids=[]
    subset_cluster_ids_app=subset_cluster_ids.append
    
    cdf_idx=range(0,len(cdf))
    
    vals_set=[]
    vals_set_app=vals_set.append
    vals_min=[]
    vals_min_app=vals_min.append
    vals_max=[]
    vals_max_app=vals_max.append
    
    for i in cdf_idx:
        vals_set_app(set(vals[i]))          
        vals_min_app(np.min(vals[i]))
        vals_max_app(np.max(vals[i]))         
    
    vmin=np.array(vals_min)
    vmax=np.array(vals_max)
    
    for i in cdf_idx:
        for j in cdf_idx:
            if(i != j):
                    #use the fact that we have ordered sets here
                    if(vmax[i]<vmin[j]):
                        break
                    elif(vmin[i]>vmax[j]):
                        break
                    else:
                        is_subset=vals_set[i].issubset(vals_set[j])
                        #print(i,j,is_subset)
                        if(is_subset):
                            subset_clusters_app(i)
                            subset_cluster_ids_app([i,j])
                            break
        if(np.mod(i,1000)==0):                
            print('Progress [%], ', i/cdf_idx[-1]*100)
        
    return subset_clusters, subset_cluster_ids    

In [ ]:
# vals=clustered_observations_final.obsId.values
# cdf_idx=range(0,len(clustered_observations_final))
# vals_set=[]
# vals_set_app=vals_set.append
# for i in cdf_idx:
#     vals_set_app(set(vals[i])) 

In [ ]:
%%time
[idx_to_drop,subset_list]=collapse_clusters(clustered_observations_final)

In [ ]:
idx_to_drop

In [ ]:
cof=clustered_observations_final.drop(index=idx_to_drop)

In [ ]:
obs_idx=[]
obs_idx_ext=obs_idx.extend
for index, row in cof.iterrows():
    #print(row.obsId)
    obs_idx_ext(row.obsId)

#cof.obsId

In [ ]:
obs_idx;

In [ ]:
cof

In [ ]:
plt.figure(dpi=150)
plt.hist(cof.obsId.values,bins=100)
plt.show()

In [ ]:
cof.obsId.values

In [ ]:
dfobs

In [ ]:
len(dfobs)

In [ ]:
obs_not_captured=set(dfobs.obsId.values)-set(obs_idx)

In [ ]:
dfobs['obj'].unique()

In [ ]:
print("objects in clusters")
for index, row in cof.iterrows():
    print(dfobs['obj'][dfobs['obsId'].isin(row.obsId)].unique())
    


In [ ]:
dfonc=dfobs[dfobs['obsId'].isin(obs_not_captured)]
dfonc

In [ ]:
clist=[]
clist_app=clist.append
for index, row in cof.iterrows():
    clist_app([dfobs['RA'][dfobs['obsId'].isin(row.obsId)].values,dfobs['DEC'][dfobs['obsId'].isin(row.obsId)].values])

In [ ]:
len(clist)

In [ ]:
clist[:][0][0]

In [ ]:
cof['obsId']

In [ ]:
# plt.figure(dpi=300)
# plt.scatter(dfobs.RA,dfobs.DEC,c=dfobs.index,s=3)
# plt.scatter(dfonc.RA,dfonc.DEC,c='r',alpha=0.5,s=1)
# plt.show()

In [ ]:
# import plotly.express as px
# fig = px.scatter(dfobs, x='RA', y='DEC',
#               color='objId')
# fig2       px.scatter(dfonc, x='RA', y='DEC',
#               color='objId')
# fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Add traces
# fig.add_trace(go.Scatter(x=dfobs.RA.values, y=dfobs.DEC.values, mode='markers', name='observations', marker_size=7,marker_color=dfobs.objId.values,marker_colorscale='Viridis'))
# fig.add_trace(go.Scatter(x=dfonc.RA.values, y=dfonc.DEC.values, mode='markers', name='obs not caught', marker_size=3))
for i in range(len(clist)):
#for i in range(10):
    fig.add_trace(go.Scatter(x=clist[:][i][0], y=clist[:][i][1], mode='markers', name='obs in clusters', marker_size=5))

fig.show()

In [ ]:
# make arrows from clusters
plt.figure(dpi=300,figsize=(12,8))
plt.xlabel('x [au]')
plt.ylabel('y [au]')
xpall=[]
vpall=[]
oid=[]
tprop=(dfobs['time'].max()+dfobs['time'].min())*0.5
i=0
for index, row in cof.iterrows():
    #print(row.obsId)
    df=dfobs[(dfobs['obsId'].isin(row.obsId))]
    #tref=(df['time'].max()+df['time'].min())*0.5
    #print(df)
    tref=tprop
    
# GENERATE ARROWS / TRACKLETS FOR THIS NIGHT
    [xarrow, varrow, tarrow]=obs2heliocentric_arrows(df,row.r,row.drdt,tref,lttc=False,
                                                     verbose=True)
    
    [xp,vp,dt] = ls.propagate_arrows_2body(xarrow,varrow,tarrow,tprop)
    
    xpall.extend(xp)
    vpall.extend(vp)
    oid.extend(np.full((np.size(xp[:,0]),1),index))
    
#     print('xarrow',xarrow)
#     print('tarrow',tarrow)
#     print('xp.shape',xp.shape)
#     print('xp',xp)
#     print('dt',dt)
    
    
#    plt.quiver(xarrow[:,0],xarrow[:,1],varrow[:,0],varrow[:,1],index,scale=1)
#    plt.quiver(xp[:,0],xp[:,1],vp[:,0],vp[:,1],color='r',scale=1,alpha=0.4)
    #plt.xlim(-2,-1)
    #plt.ylim(-3,1)

plt.show()    

In [ ]:
oid

In [ ]:
xpv=np.array(xpall)


In [ ]:
xpv

In [ ]:
plot_positions3d(oid,xpv[:,0],xpv[:,1],xpv[:,2],xlim=[-3,3],ylim=[-3,3])

In [ ]:
def jac_rosenbrock(x):
    return np.array([
        [-20 * x[0], 10],
        [-1, 0]])

def fun_rosenbrock(x):
    return np.array([10 * (x[1] - x[0]**2), (1 - x[0])])

from scipy.optimize import least_squares
x0_rosenbrock = np.array([2, 2])
res_2 = least_squares(fun_rosenbrock, x0_rosenbrock, jac_rosenbrock,
                      bounds=([-np.inf, 1.5], np.inf),loss='cauchy')

res_2.x

res_2.cost

res_2.optimality

In [ ]:
def jac_radec2xyz(x,xobs=xobs)
    #
    dx=x-xobs
    dradx=-dx[:,1]/(dx[:,0]**2+dy[:,0]**2)
    drady=dx[:,0]/(dx[:,0]**2+dy[:,0]**2)
    dradz=0
    
    ddecdx=-dx[:,]

In [ ]:
# Gaussian Gravitational Constant [au^1.5/Msun^0.5/D]
gaussk = 0.01720209894846
# default gravitational parameter [Gaussian units]
gm = gaussk*gaussk

In [ ]:
x=np.array([1,0,0])
v=np.array([0,gaussk,0])

In [ ]:
dt=90

In [ ]:
res=sp.prop2b(gm, np.hstack((x, v)), dt)

In [ ]:
res

In [ ]:
org=sp.prop2b(gm, res, -dt)

In [ ]:
org

## DIFI output

In [ ]:
#clustered_observations_final.to_csv('LSST_SSP_DIFI_NONOISE_3days.txt',sep=' ',index=False)


In [ ]:
#dfobs.to_csv('VC_NONOISE_3days.obs',sep=' ',index=False)

In [ ]:
#observations in each cluster
# obs_in_cluster, labels = observations_in_cluster(dfobs,obsids,db)

# obs_in_cluster_df=pd.DataFrame(zip(labels,obs_in_cluster),columns=['clusterId','obsId'])

## Plots

In [ ]:
# #object(s) in each cluster
# obj_in_cluster, labels=objects_in_cluster(dfobs,objids,db)

# labels

# obj_in_cluster_df=pd.DataFrame(zip(labels,obj_in_cluster), columns=['clusterId','objId'])

# obj_in_cluster_df

Plot the arrow positions for the test asteroids

In [ ]:
no=20
plt.figure(dpi=150,figsize=(12,8))
plt.quiver(xarrow[:no,0],xarrow[:no,1],varrow[:no,0],varrow[:no,1],range(no),scale=2)
plt.xlim(-2,-1)
plt.ylim(-3,1)
plt.show()

In [ ]:
xpall[2]

In [ ]:
plt.figure(dpi=150,figsize=(12,8))
#plt.quiver(xp[:no,0],xp[:no,1],vp[:no,0],vp[:no,1],range(no),scale=0.6)
#plt.xlim(0,0.25)
# plt.xlim(-1.6,-1.2)
# plt.ylim(-2,-1.8)
rng=range(len(xpall))
#rng=range(100)
for i in rng:
    plt.quiver(xpall[i][:,0],xpall[i][:,1],vpall[i][:,0],vpall[i][:,1],rng,scale=1,alpha=0.5)

plt.colorbar()
plt.show()

In [ ]:
plt.figure(dpi=150,figsize=(12,8))
plt.scatter(xp[:no,0],xp[:no,1],s=0.3)
plt.show()

In [ ]:
# def create_heliocentric_arrows(df, r, drdt, tref, cr, ct_min, ct_max, v_max=1., lttc=False, filtering=True, verbose=True, eps=0):
#     """Create tracklets/arrows from dataframe containing nightly RADEC observations
#        and observer positions.

#     Parameters:
#     -----------
#     df       ... Pandas DataFrame containing nightly RA and DEC [deg], time [JD, MJD],
#                  (x,y,z)_observer positions [au, ICRF]
#     r        ... assumed radius of heliocentric sphere used for arrow creation[au]
#     drdt     ... assumed radial velocity
#     tref     ... reference time for arrow generation. Used to calculate how much the 
#                  heliocentric distance changes between observations based on assumed dr/dt
#     cr       ... maximum spacial clustering radius for arrow creation (au)
#     ct_min   ... minimum temporal clusting radius for arrow creation (days)
#     ct_max   ... maximum temporal clusting radius for arrow creation (days)


#     Keyword arguments:
#     ------------------
#     v_max (optional)       ... velocity cutoff [au/day]
#     lttc (optional)        ... light travel time correction
#     filtering (optional)   ... filter created tracklets (exclude tracklets built 
#                                 from data with the same timestamp) 
#     verbose (optional)     ... print verbose progress statements  
#     eps (optional)         ... Branches of the Kdtree are not explored if their 
#                                nearest points are further than r/(1+eps), 
#                                and branches are added in bulk if their furthest points 
#                                are nearer than r * (1+eps). eps has to be non-negative.

#     Returns:
#     --------
#     x         ... tracklet/arrow position (3D) [au]
#     y         ... tracklet/arrow velocity (3D) [au]
#     t         ... tracklet/arrow reference epoch [JD/MJD]
#     goodpairs ... index pairs of observations that go into each tracklet/arrow
#     """
    
#     goodpairs=[]
#     paris=[]
    
#     # speed of light in au/day
#     c_aupd = 173.145

#     # Transform RADEC observations into positions on the unit sphere (US)
#     xyz = ls.radec2icrfu(df['RA'], df['DEC'], deg=True)

#     # Those are the line of sight (LOS) vectors
#     los = np.array([xyz[0], xyz[1], xyz[2]]).T

#     # Use the position of the observer and the LOS to project the position of
#     # the asteroid onto a heliocentric great circle with radius r
#     observer = df[['x_obs', 'y_obs', 'z_obs']].values

#     # Calculate how much the heliocentric distance changes
#     # during the obsevations based on assumed dr/dt
#     dt = tref-df['time'].values
#     dr = drdt*dt
#     r_plus_dr = r+dr

#     # Heliocentric postions of the observed asteroids
#     posu = ls.sphere_line_intercept(los, observer, r_plus_dr)

#     if(verbose):
#         print('Heliocentric positions generated.')
#         print('Building spacial KDTree...')
        
#     # To generate tracklets we build our KDTree based on the positions
#     # in heliocentric space
#     kdtree_s = scsp.cKDTree(posu, leafsize=16, compact_nodes=True,
#                           copy_data=False, balanced_tree=True, boxsize=None)
#     # rule out un-physical combinations of observations with kdtree

#     # Query KDTree for good pairs of observations that lie within
#     # the clustering radius cr
#     if(verbose):
#         print('KDTree generated. Creating tracklets...')
        
#     pairs = kdtree_s.query_pairs(cr,p=2., eps=eps, output_type='ndarray')

#     if(verbose):
#         print('Tracklet candidates found:',len(pairs))

#     if (filtering):
#         if(verbose):
#             print('Filtering arrows by time between observations...')
        
#         # Discard impossible pairs (same timestamp)
#         [df2, goodpairs] = ls.SelectTrackletsFromObsData(pairs, df, ct_min, ct_max, 'time')
        
#         if(verbose):
#             print('Tracklets filtered. New number of tracklets:',len(goodpairs))
    
#     else:
#         goodpairs=pairs
    
    
#     # tracklet position for filtered pairs
#     x = posu[goodpairs[:,0]]
#     # tracklet time
#     t = df['time'][goodpairs[:,0]].values
#     # tracklet velocity through forward differencing
#     va = []
#     vapp = va.append
#     dt = df['time'][goodpairs[:,1]].values-df['time'][goodpairs[:,0]].values
#     dx = posu[goodpairs[:,1]]-posu[goodpairs[:,0]]
#     for d in range(0,3):
#         vapp(np.divide(dx[:,d],dt))
#     v = np.array(va).T
    
#     if (filtering):
#         if(verbose):
#             print('Filtering arrows by max velocity...')
#         vnorm=ls.norm(v)
#         v_idx=np.where(vnorm<=v_max)[0]
    
#         goodpairs=np.take(goodpairs,v_idx,axis=0)
#         x=np.take(x,v_idx,axis=0)
#         v=np.take(v,v_idx,axis=0)
#         t=np.take(t,v_idx,axis=0)
    
# #         print('lenx_filtered',len(x))
# #         print('lenv_filtered',len(v))
# #         print('lent_filtered',len(t))
# #         print('x',x)
# #         print('v',v)
#         print('t',t)
# #         print('goodpairs',goodpairs)
    
#     if(verbose):
#         print('Tracklets created:',len(goodpairs))
    
#     # correct arrows for light travel time
#     if(lttc):
#         if(verbose):
#             print('(Linear correction for light travel time aberration...')
#         xo = observer[goodpairs[:, 0]]
#         dist = ls.norm(x-xo)
#         xl = x.T-dist/c_aupd*v.T
#         return xl.T, v, t, goodpairs

#     else:
#         return x, v, t, goodpairs

In [ ]:
plt.figure(dpi=150,figsize=(12,8))
plt.scatter(xp[:no,0],xp[:no,2],c=range(no),s=0.3)
#plt.xlim(0,0.25)
plt.xlim(0,1)
plt.ylim(-2,2)
plt.show()

In [ ]:
plot_positions3d(objids[:no,0],xp[:no,0], xp[:no,1], xp[:no,2],xlim=[-2,2],ylim=[-2.5,2])

In [ ]:
import chart_studio.plotly as py

data = [{
    'type': 'cone',
    'x': xp[:no,0], 'y': xp[:no,1], 'z': xp[:no,2],
    'u': vp[:no,0], 'v': vp[:no,1], 'w': vp[:no,2]
}]

layout = {
    'scene': {
      'camera': {
        'eye': {'x': -0.76, 'y': 1.8, 'z': 0.92}
      }
    }
}

fig = {"data": data, "layout": layout}
py.iplot(fig, filename='cone-basic', validate=False)

In [ ]:
dt=(tobs[:,1]-tobs[:,0])*24

In [ ]:
plt.figure(dpi=300,figsize=(12,12))
plt.xlabel('time between observations that form  a tracklet [hrs]')
plt.ylabel('counts')
plt.yscale('log')
plt.xlim(0,10)
plt.hist(dt,bins=1000)

plt.show()

In [ ]:
#Plot selected observations
plt.figure(dpi=300,figsize=(12,12))

plt.scatter(dfobs['RA'].values[obsids],dfobs['DEC'].values[obsids],s=0.1,c=dfobs['objId'].values[obsids])
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')
plt.ylim(-45,-40)
plt.xlim(190,200)
plt.colorbar()
plt.show()

In [ ]:
plot_positions3d(objids[:200000,0],xp[:200000,0], xp[:200000,1], xp[:200000,2],xlim=[-2,2],ylim=[-2,2])

In [ ]:
plot_positions3d(objids[:100000,0],vp[:100000,0], vp[:100000,1], vp[:100000,2],xlim=[-2,2],ylim=[-2,2])

### Now propagate the arrows to the same time (the center of the t array)

In [ ]:
[xp,vp,dt] = ls.propagate_arrows_2body(xarrow,varrow,tarrow,df['time'].median())

In [ ]:
df['time'].median()

In [ ]:
plot_positions3d(dt[:100000],xp[:100000,0], xp[:100000,1], xp[:100000,2],xlim=[-2,2],ylim=[-2,2])

### Let's cluster!

In [ ]:
#use astrometric uncertainty to determine clustering error (150mas (3sigma) )
astrometric_uncertainty=300/1000/3600/180*np.pi

# DBSCAN clustering parameters
eps=astrometric_uncertainty*r
eps=0.002
min_samples=3

In [ ]:
eps

In [ ]:
# Create 6D array of tracklets
# xpvp=np.hstack((xp,varrown))
xpvp=np.hstack((xp,vp/np.median(vp)))

In [ ]:
#Scale arrow speed to unit lenght to facilitate clustering in 6D space
# vn=np.linalg.norm(varrow,axis=1)
# varrown=[v/np.linalg.norm(v) for v in varrow]

Cluster in propagated position space (3D), Black dots are not clustered (garbage)

In [ ]:
plot_clusters3d(xp, [0,1,2], cluster.DBSCAN, (), {'eps':eps,'min_samples':min_samples})

Cluster in propagated position and velocity space (6D)


In [ ]:
# Heliocentric position space clustering
plot_clusters3d(xpvp,[0,1,2], cluster.DBSCAN, (), {'eps':eps,'min_samples':min_samples})

In [ ]:
# Velocity clustering
plot_clusters3d(xpvp,[3,4,5], cluster.DBSCAN, (), {'eps':eps,'min_samples':min_samples})

## Try multiple heliocentric radii for clustering

In [ ]:
# range of radii
rall=np.arange(0.8,4,0.4)

# clustering radius for trackelt creation based on how far an object can move during one night 
dt=1
cr=50/150e6*86400*dt

# only one raidal velocity value [au/d]
drdt=0.001

# reference epoch for propagation
tref=df['time'].median()


xpall=[]
vpall=[]
dtall=[]

for r in rall:
    [xarrow, varrow, tarrow, goodpairs] = ls.create_heliocentric_arrows(df,r,drdt,tref,cr)
    [xp,vp,dt] = ls.propagate_arrows_2body(xarrow,varrow,tarrow,df['time'].median())
    
    xpall.append(xp)
    vpall.append(vp)
    dtall.append(dt)

Plot the propagated arrows as for each clustering radius. The red arrows show the actual states of the test asteroids. The blue circle represents the Earth's orbit.

In [ ]:
plt.figure(dpi=150,figsize=(12,12))

plt.scatter(0,0,s=4,c='r')
plt.xlabel('x [au]')
plt.ylabel('y [au]')

circle1=plt.Circle((0, 0), 1, color='b', fill=False)

#for i in range(len(rall)):   
plt.quiver(xp[i][:,0],xp[i][:,1],vp[i][:,0],vp[i][:,1],scale=0.7)


# for i in range(len(asteroids)):
#     plt.quiver(horizon_states[i][0][5],horizon_states[i][0][6],horizon_states[i][0][8],horizon_states[i][0][9],color='red')
    
plt.xlim(-4,4)
plt.ylim(-4,4)

plt.gcf().gca().add_artist(circle1)
plt.show()

In [ ]:
for i in range(len(rall)):  
    print('heliocentric distance sampled [au]:', rall[i])
    
    #use astrometric uncertainty to determine clustering error (150mas (3sigma) )
    astrometric_uncertainty=3*50/1000/3600/180*np.pi

    # DBSCAN clustering parameters
    eps=astrometric_uncertainty*rall[i]
    
    plot_clusters3d(xpall[i], [0,1,2], cluster.DBSCAN, (), {'eps':eps,'min_samples':min_samples})

### And in numbers ... 

In [ ]:
# r=2.6
# drdt=0.001
# tref=df['time'].median()
# cr=0.1

# #use astrometric uncertainty to determine clustering error (150mas (3sigma) )
# astrometric_uncertainty=3*50/1000/3600/180*np.pi

# # DBSCAN clustering parameters
# eps=astrometric_uncertainty*r

# [xarrow, varrow, tarrow, goodpairs] = ls.create_heliocentric_arrows(df,r,drdt,tref,cr)
# [xp,vp,dt] = ls.propagate_arrows_2body(xarrow,varrow,tarrow,df['time'].median())



In [ ]:
#use astrometric uncertainty to determine clustering error (150mas (3sigma) )
astrometric_uncertainty=300/1000/3600/180*np.pi

# DBSCAN clustering parameters
eps=astrometric_uncertainty*r
eps=0.003
min_samples=3

%%time
db=cluster.DBSCAN(eps=eps,min_samples=min_samples,n_jobs=4).fit(xp)

In [ ]:
%%time
db=cluster.DBSCAN(eps=eps,min_samples=min_samples,n_jobs=4).fit(xp)
# db=cluster.hdbscan.HDBSCAN(algorithm='best', alpha=1.0, approx_min_span_tree=True,
#     gen_min_span_tree=True, leaf_size=40, memory=Memory(cachedir=None),
#     metric='euclidean', min_cluster_size=3, min_samples=None, p=None, cluster_selection_epsilon=eps,cluster_selection_method='leaf').fit(xp)

In [ ]:
eps

In [ ]:
clusters=np.unique(db.labels_)

In [ ]:
goodpairs=obsids

In [ ]:
df=dfobs

In [ ]:
df;

In [ ]:
clusters

In [ ]:
#Calculate metrics
metric=metrics(dfobs,goodpairs,db)

In [ ]:
# Number of objects that ought to have been observed 
metric[0]

In [ ]:
# Number of clusters (including garbage), 
metric[1]

In [ ]:
# number of perfect clusters (all obs of a given obj in one cluster) 
metric[2]

In [ ]:
# number of pure clusters (obs of only one object in a cluster if not perfect)
metric[3]

In [ ]:
# number of impure clusters (obs of multiple objects in one cluster)
metric[4]

In [ ]:
#Completeness [%] (fraction of all observed objects that have been collected into clusters)
metric[5]

In [ ]:
#Missing objects
metric[6].compressed()

In [ ]:
# cluster labels: -1 is the garbage cluster
#metrics(df,goodpairs,db)[7]

### Observations in each arrow

In [ ]:
# Find observations in each arrow
dfoia=observations_in_arrows(dfobs,goodpairs,columns=['obs_t0','obs_t1'])

In [ ]:
# Find corresponding entry in obs database
dfobs.iloc[dfoia['obs_t0'][0]]

### Observations in each cluster

In [ ]:
#observations in each cluster
obs_in_cluster, labels = observations_in_cluster(dfobs,goodpairs,db)

In [ ]:
obs_in_cluster_df=pd.DataFrame(zip(labels,obs_in_cluster),columns=['clusterId','obsId'])

In [ ]:
obs_in_cluster_df

In [ ]:
obs2difi=df2difi(obs_in_cluster_df,'clusterId','obsId')

In [ ]:
obs2difi.to_csv('obs_per_cluster.csv',index=False)

### Objects in each cluster

In [ ]:
#object(s) in each cluster
obj_in_cluster, labels=objects_in_cluster(dfobs,objid,db)

In [ ]:
labels

In [ ]:
obj_in_cluster_df=pd.DataFrame(zip(labels,obj_in_cluster), columns=['clusterId','objId'])

In [ ]:
obj_in_cluster_df

In [ ]:
obj2difi=df2difi(obj_in_cluster_df,'clusterId','objId')

In [ ]:
obj2difi.to_csv('obj_per_cluster.csv',index=False)